# Groundwater Flow example

This example is taken from Beskos et al (2016) 'Geometric MCMC for infinite-dimensional inverse problems', and the notation largely follows their section 4, and we commented where we deviated from it.

In [1]:
import time
import numpy as np
import random
import sys
from scipy.stats import norm
from copy import deepcopy as dc

import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
import numba

sigma = 0.01                        # noise, called sigma_y in the cited paper
dim = 2                             # dimensionality of the space (v: R^d --> R)
obs_min = np.array([0,0])
obs_max = np.array([1,1])

In [2]:
"""

initialise KL

"""

alpha = 1.1 #called s in the cited paper, as their alpha is 0, we omitted it here, as well as their sigma=1
def eigenvalues():
    ev = np.zeros((N_trunc,N_trunc))
    ev_root = np.zeros((N_trunc,N_trunc))
    for i in range(N_trunc):
        for j in range(N_trunc):
            ev[i,j] = 1/(np.pi**2*((i+1/2)**2+(j+1/2)**2))**alpha
    ev_root = ev**(1/2)
    return ev,ev_root

In [3]:
"""

FUNCTIONS - PRIOR

"""

def sample_prior(c=1):
    xi = np.zeros((N_trunc,N_trunc))
    for i in range(N_trunc):
        for j in range(N_trunc):
            xi[i,j] = ev_root[i,j]*np.random.normal()
    return xi*c

def logprior(xi):
    logprior = 0
    for i in range(N_trunc):
        for j in range(N_trunc):
            logprior += np.log(norm._pdf(xi[i,j]/ev_root[i,j]))
    return logprior

In [4]:
"""

FUNCTIONS - LIKELIHOOD 

"""
    
m = 21

# Define h where it is fixed (when x_2=0 or x_2=1).
h_first_row = np.zeros(m)
h_last_row = np.zeros(m)
for i in range(m): # Points on the bottom, where x_2=0 (h=x_1)
    h_first_row[i] = i/(m-1)
for i in range(m): # Points on top, where x_2=1 (h=1-x_1)
    h_last_row[i] = 1-i/(m-1)  
    
"""
Initialise the finite difference scheme.
"""
@numba.njit()
def A_init(k):   
    A = np.zeros((m**2,m**2))
    for j in range(m-2): # central points
        for i in range(m-2):
            A[(i+1)+m*(j+1),(i+1)+m*(j+1)] = 2*k[(i+1)+m*(j+1)]+k[(i)+m*(j+1)]+k[(i+1)+m*(j)]
            A[(i+1)+m*(j+1),(i)+m*(j+1)] = -k[(i)+m*(j+1)]
            A[(i+1)+m*(j+1),(i+1)+m*(j)] = -k[(i+1)+m*(j)]
            A[(i+1)+m*(j+1),(i+2)+m*(j+1)] = -k[(i+1)+m*(j+1)]
            A[(i+1)+m*(j+1),(i+1)+m*(j+2)] = -k[(i+1)+m*(j+1)]
    i = m-1 # Points on the right, where x=6 (h_x=0)
    for j in range(m-2):
        A[i+m*(j+1),i+m*(j+1)] = k[i+m*(j+1)]+k[i+m*(j)]+k[i-1+m*(j+1)]
        A[i+m*(j+1),i+m*(j)] = -k[i+m*(j)]
        A[i+m*(j+1),i+m*(j+2)] = -k[i+m*(j+1)]
        A[i+m*(j+1),i-1+m*(j+1)] = -k[i-1+m*(j+1)]
    i = 0 # Points on the left, where x=0 (h_x=0)
    for j in range(m-2):
        A[i+m*(j+1),i+m*(j+1)] = 2*k[i+m*(j+1)]+k[i+m*(j)]
        A[i+m*(j+1),i+m*(j)] = -k[i+m*(j)]
        A[i+m*(j+1),(i+1)+m*(j+1)] = -k[i+m*(j+1)]
        A[i+m*(j+1),i+m*(j+2)] = -k[i+m*(j+1)]       
    # Corner points are already considered in the cases where x_2=0 and x_2=1.'''   
    A = 1/((6/(m-1))**2)*A   # Multiply by 1/Delta^2
    return A

"""
G maps a function u to the output of a PDE, h. It makes use of the finite 
difference approximation given by A.k and b_k. It's specific for the problem
setup.
"""
@numba.njit()
def G(k):    
    A_k = A_init(k)
    h = np.zeros(m**2)
    b = np.zeros(m**2)
    for i in range(m):
        b[i+m] -= A_k[i+m,i]*h_first_row[i]
        b[i+m*(m-2)] -= A_k[i+m*(m-2),i+m*(m-1)]*h_last_row[i]        
        
    # As the first and last column are fixed, we only consider a subset of the problem for the linalg solver.
    h_sub = np.zeros(m*(m-2))     
    A_sub = A_k[:,m:m*(m-1)]
    A_sub = A_sub[m:m*(m-1),:]
    h_sub = np.linalg.solve(A_sub,b[m:m*(m-1)])

    h[m:m*(m-1)] = h_sub
    h[0:m] = h_first_row
    h[m*(m-1):] = h_last_row    
    return h

def loglikelihood(xi,y):
    k = np.zeros(m**2)
    for j in range(m):
        for i in range(m):
            k[i+m*j] = np.exp(u(xi,(i/(m-1),j/(m-1))))
    h = G(k)
    y_h = h[obs_indices]
    LL = -1/sigma**2*np.dot(y_h-y,y_h-y)/2
    return LL

In [5]:
"""

FUNCTIONS - VALUE FUNCTION AND POLICIES

"""

''' u(xi,x), which evaluates the function u with coefficients xi at x=(pos,speed) --- new, complicated, but fast version '''
@numba.njit()
def u(xi,x):
    u_sum = 0
    for i in range(N_trunc):
        for j in range(N_trunc):
            i_eval = np.pi*(i+1/2)*x[0]
            j_eval = np.pi*(j+1/2)*x[1]
            u_sum += xi[i,j]*np.cos(i_eval)*np.cos(j_eval)
    return 2*u_sum

In [6]:
"""

FUNCTIONS - ANALYTICS

"""
    
''' Progress bar to know how much longer one has to wait '''
def progressBar(t,value, t_max, acceptances, bar_length=40):
    percent = float(t) / t_max
    arrow = '-' * int(round(percent * bar_length)-1) + '>'
    spaces = ' ' * (bar_length - len(arrow))
    sys.stdout.write("\rIteration: {0}    Acceptance ratio: {1}    Percent: [{2}] {3}%  ".format(value,round(acceptances/value,3),arrow + spaces, int(round(percent * 100))))
    sys.stdout.flush()     
        
''' Plotting a value function '''    
def func_plot(xi,name):
    x = np.arange(0,1,0.005)
    y = np.arange(0,1,0.005)
    X,Y = np.meshgrid(x,y)
    Z = np.zeros(X.shape)
    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            x=np.zeros(dim)
            x[0]=X[i,j]
            x[1]=Y[i,j]
            Z[i,j] = np.exp(u(xi,x))

    fig, ax = plt.subplots()
    c = ax.pcolormesh(X, Y, Z, cmap='cool', vmin=Z.min(), vmax=Z.max())

    # set the limits of the plot to the limits of the data
    ax.axis([X.min(), X.max(), Y.min(), Y.max()])
    fig.colorbar(c, ax=ax)
    fig.savefig(name + '.pdf', dpi=300, bbox_inches='tight')
    plt.close(fig)
    
''' Plot a trajectory '''  
def trajectory_plot(xi,name):
    x = np.arange(len(xi))
    fig = plt.figure()
    plt.plot(x,xi)
    fig.savefig(name + '.png', bbox_inches='tight')
    plt.close(fig)
    
''' Compute the autocorrelations '''    
def autocorr(x,lags):
    mean=np.mean(x)
    var=np.var(x)
    xp=x-mean
    corr=[1. if l==0 else np.sum(xp[l:]*xp[:-l])/len(x)/var for l in lags]
    return np.array(corr)

''' Calculate the Effective Sample Size, assumes algorithm already burned in '''
def ESS(logposterior,name):
    fig, ax = plt.subplots()
    N = len(logposterior)
    ax.stem(autocorr(logposterior, range(int(N*0.1))),use_line_collection=True) 
    ESS = N/(1+2*sum(autocorr(logposterior, range(int(N*0.1)))))
    print('\nEffective Sample Size:', round(ESS))
    print('Samples required to generate 1 independent sample:', round(N/ESS,2))
    fig.savefig(name + '.png', bbox_inches='tight')
    plt.close(fig)    

In [7]:
"""

FUNCTIONS - MCMC (pCN/pCNL)

"""

def acceptance_prop(xi_u, xi_v,data,ll_u,diff_u=False):
    accept_prop = -ll_u
    ll_v = loglikelihood(xi_v,data)
    
    accept_prop += ll_v
        
    return min(1, np.exp(accept_prop)),ll_v
    
def propose(xi,diff=False):
    proposal = np.sqrt(1-beta*beta)*xi+beta*sample_prior()
    return proposal

def MCMC(xi,N_data,data,max_time):   
    print('\nMCMC algorithm ('+method + ', N_trunc=' + str(N_trunc) + ', N_data=' + str(N_data) + ', ' + str(max_time) + ' seconds) was started: ' + str(time.ctime()))
        
    acc_ratio = 0
    logposterior = []
    logp = []
    logl = []
    
    ''' Initialise likelihood and gradient '''  
    ll = loglikelihood(xi,data)
    print('Initial loglikelihood: ',ll)
    
    ''' Run MCMC '''
    start = time.time() 
    j = 0
    it = 0
    while(time.time()-start<max_time):
        
        ''' Propose and calculate acceptance probability '''
        if method=='CNL' or method=='pCNL':
            xi_proposal = propose(xi,diff)  
            a,ll_proposal,diff_proposal = acceptance_prop(xi,xi_proposal,data,ll,diff)
        else:
            xi_proposal = propose(xi)  
            a,ll_proposal = acceptance_prop(xi,xi_proposal,data,ll)
        
        ''' Accept or reject proposal '''
        uni = np.random.uniform()
        if uni < a:
            xi = xi_proposal    
            ll = ll_proposal
            acc_ratio = acc_ratio + 1
            

        ''' prior, likelihood, and posterior traceplots are appended '''
        lp = logprior(xi)
        logposterior.append(lp+ll)
        logp.append(lp)
        logl.append(ll)
        
        if (time.time()-start)>it*t_max/8000 and it<8000:
            ''' store sample for future use '''
            np.save('np_saved/GWF/samples_policy_learning/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_sampleNo'+str(it)+'.npy',xi)
            it += 1
        
        if (j+1)%100==0:
            progressBar(time.time()-start,j+1,max_time,acc_ratio)
        j+=1
        
    progressBar(max_time,j,max_time,acc_ratio)
    
    acc_ratio = acc_ratio/(j)
    print('\nMCMC algorithm terminated: ' + str(time.ctime()) + '. \nRuntime = ' + str(time.time()-start))
    print('Final loglikelihood: ',ll)
    print('Acceptance ratio is ',acc_ratio)
    
    trajectory_plot(logposterior[1:],'figs/GWF/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_logposterior')
    trajectory_plot(logp[1:],'figs/GWF/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_logprior')
    trajectory_plot(logl[1:],'figs/GWF/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_loglikelihood')

    np.save('np_saved/GWF/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_lastSample.npy',xi)
    func_plot(xi,'figs/GWF/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_lastSample')
    
    ESS(logposterior,'figs/GWF/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_autocorr')    

# MAIN PROGRAMMES

In [8]:
"""

MAIN PROGRAMME

"""       

# set maximal runtime
t_max = 4*24*(3600/10)   # roughly whatever NN does times 2/23

N_trunc = 25
ev,ev_root = eigenvalues()

# Sample from the prior to see what a sample looks like
xi = sample_prior()
func_plot(xi,'figs/GWF/KL_'+str(N_trunc)+'_a_prior_sample')

N_data = 33
data = np.load('GWF_data.npy')
obs_indices = np.load('GWF_obs_indices.npy')

''' run pCN '''
method = 'pCN'
beta =  1/7.5
try:
    xi = np.load('np_saved/GWF/KL_'+str(N_trunc)+'_'+method+'_NData'+str(N_data)+'_lastSample.npy')
except FileNotFoundError:
    print('Starting from close to 0')
    xi = sample_prior(0.1)
    
MCMC(xi,N_data,data,t_max) 


MCMC algorithm (pCN, N_trunc=25, N_data=33, 34560.0 seconds) was started: Sun Aug 16 21:20:50 2020
Initial loglikelihood:  -14.69742538464267
Iteration: 4522119    Acceptance ratio: 0.232    Percent: [--------------------------------------->] 100%  
MCMC algorithm terminated: Mon Aug 17 06:56:52 2020. 
Runtime = 34560.00128722191
Final loglikelihood:  -22.9576893340097
Acceptance ratio is  0.23163079078635482

Effective Sample Size: 3492.0
Samples required to generate 1 independent sample: 1294.85
